In [24]:
import numpy as np
import pandas as pd
import sqlalchemy as sa
import os

from dotenv import load_dotenv

In [2]:
# b_data is short for business dataset
# lines = True helps with the Trailing data ValueError

b_data = pd.read_json('../../data_projects/data/yelp/yelp_dataset/yelp_academic_dataset_business.json', lines = True)

In [3]:
# Inspecting the data

b_data.head()

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
0,Pns2l4eNsfO8kk83dixA6A,"Abby Rappoport, LAC, CMQ","1616 Chapala St, Ste 2",Santa Barbara,CA,93101,34.426679,-119.711197,5.0,7,0,{'ByAppointmentOnly': 'True'},"Doctors, Traditional Chinese Medicine, Naturop...",None
1,mpf3x-BjTdTEA3yCZrAYPw,The UPS Store,87 Grasso Plaza Shopping Center,Affton,MO,63123,38.551126,-90.335695,3.0,15,1,{'BusinessAcceptsCreditCards': 'True'},"Shipping Centers, Local Services, Notaries, Ma...","{'Monday': '0:0-0:0', 'Tuesday': '8:0-18:30', ..."
2,tUFrWirKiKi_TAnsVWINQQ,Target,5255 E Broadway Blvd,Tucson,AZ,85711,32.223236,-110.880452,3.5,22,0,"{'BikeParking': 'True', 'BusinessAcceptsCredit...","Department Stores, Shopping, Fashion, Home & G...","{'Monday': '8:0-22:0', 'Tuesday': '8:0-22:0', ..."
3,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,935 Race St,Philadelphia,PA,19107,39.955505,-75.155564,4.0,80,1,"{'RestaurantsDelivery': 'False', 'OutdoorSeati...","Restaurants, Food, Bubble Tea, Coffee & Tea, B...","{'Monday': '7:0-20:0', 'Tuesday': '7:0-20:0', ..."
4,mWMc6_wTdE0EUBKIGXDVfA,Perkiomen Valley Brewery,101 Walnut St,Green Lane,PA,18054,40.338183,-75.471659,4.5,13,1,"{'BusinessAcceptsCreditCards': 'True', 'Wheelc...","Brewpubs, Breweries, Food","{'Wednesday': '14:0-22:0', 'Thursday': '16:0-2..."


In [4]:
# Inspecting the data

b_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150346 entries, 0 to 150345
Data columns (total 14 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   business_id   150346 non-null  object 
 1   name          150346 non-null  object 
 2   address       150346 non-null  object 
 3   city          150346 non-null  object 
 4   state         150346 non-null  object 
 5   postal_code   150346 non-null  object 
 6   latitude      150346 non-null  float64
 7   longitude     150346 non-null  float64
 8   stars         150346 non-null  float64
 9   review_count  150346 non-null  int64  
 10  is_open       150346 non-null  int64  
 11  attributes    136602 non-null  object 
 12  categories    150243 non-null  object 
 13  hours         127123 non-null  object 
dtypes: float64(3), int64(2), object(9)
memory usage: 16.1+ MB


In [5]:
# Make a copy to allow for easy editing

for_sql = b_data.copy()

## Cleaning the `hours` Column.

Because this column is given as JSON, we will need to apply `pd.json_normalize` to the `hours` column, clean the data, then concatenate the columns with the existing DataFrame.

In [6]:
# Create a DataFrame of the business hours
# Will be used to concat alongside for_sql DataFrame

hours_df = pd.json_normalize(for_sql['hours'].to_list())

In [7]:
# Viewing the DataFrame

hours_df

,Monday,Tuesday,Wednesday,Thursday,Friday,Saturday,Sunday
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0:0-0:0,8:0-18:30,8:0-18:30,8:0-18:30,8:0-18:30,8:0-14:0,NaN
2,8:0-22:0,8:0-22:0,8:0-22:0,8:0-22:0,8:0-23:0,8:0-23:0,8:0-22:0
3,7:0-20:0,7:0-20:0,7:0-20:0,7:0-20:0,7:0-21:0,7:0-21:0,7:0-21:0
4,NaN,NaN,14:0-22:0,16:0-22:0,12:0-22:0,12:0-22:0,12:0-18:0
...,...,...,...,...,...,...,...
150341,10:0-19:30,10:0-19:30,10:0-19:30,10:0-19:30,10:0-19:30,10:0-17:30,11:0-17:0
150342,9:30-17:30,9:30-17:30,9:30-17:30,9:30-17:30,9:30-17:30,9:0-17:0,0:0-16:0
150343,NaN,NaN,NaN,NaN,NaN,NaN,NaN
150344,9:0-20:0,9:0-20:0,9:0-20:0,9:0-20:0,9:0-20:0,9:0-17:0,10:0-17:0


### Note About Cleaning the `hours` Column

Because times given as `0:0` could represent an opening or closing time of midnight, if we treat these as `NaN` values, we could lose information about companies that open or close at midnight, or are open for 24 hours. To work around this, we will make sure that the `type` of the data in the cell is not a `float`, and alter the data accordingly.

In [8]:
# Create a opening time and closing time column

hours_df['monday_opening'] = hours_df['Monday'].apply(
    lambda x: x.split('-')[0] if type(x) is not float else np.nan
)
hours_df['monday_closing'] = hours_df['Monday'].apply(
    lambda x: x.split('-')[1] if type(x) is not float else np.nan
)

hours_df['tuesday_opening'] = hours_df['Tuesday'].apply(
  lambda x: x.split('-')[0] if type(x) is not float else np.nan
)
hours_df['tuesday_closing'] = hours_df['Tuesday'].apply(
  lambda x: x.split('-')[1] if type(x) is not float else np.nan
)

hours_df['wednesday_opening'] = hours_df['Wednesday'].apply(
  lambda x: x.split('-')[0] if type(x) is not float else np.nan
)
hours_df['wednesday_closing'] = hours_df['Wednesday'].apply(
  lambda x: x.split('-')[1] if type(x) is not float else np.nan
)

hours_df['thursday_opening'] = hours_df['Thursday'].apply(
  lambda x: x.split('-')[0] if type(x) is not float else np.nan
)
hours_df['thursday_closing'] = hours_df['Thursday'].apply(
  lambda x: x.split('-')[1] if type(x) is not float else np.nan
)

hours_df['friday_opening'] = hours_df['Friday'].apply(
  lambda x: x.split('-')[0] if type(x) is not float else np.nan
)
hours_df['friday_closing'] = hours_df['Friday'].apply(
  lambda x: x.split('-')[1] if type(x) is not float else np.nan
)

hours_df['saturday_opening'] = hours_df['Saturday'].apply(
  lambda x: x.split('-')[0] if type(x) is not float else np.nan
)
hours_df['saturday_closing'] = hours_df['Saturday'].apply(
  lambda x: x.split('-')[1] if type(x) is not float else np.nan
)

hours_df['sunday_opening'] = hours_df['Sunday'].apply(
  lambda x: x.split('-')[0] if type(x) is not float else np.nan
)
hours_df['sunday_closing'] = hours_df['Sunday'].apply(
  lambda x: x.split('-')[1] if type(x) is not float else np.nan
)

In [9]:
# Drops the selected columns
# No longer needed because this information is given in opening and closing time columns

hours_df.drop(
    columns = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday'],
    axis = 1,
    inplace = True
)

In [10]:
# This is how we will check if the time is improperly formatted

hours_df['monday_opening'][1][-2:]

':0'

In [11]:
# If the time is improperly formatted, add a 0 to the end

def clean_time(time_str):
    if type(time_str) is not float and time_str[-2:] == ':0':
        time_str = time_str + '0'
    return time_str    

In [12]:
# Apply the transformation, then check the DataFrame

for col in (hours_df.columns):
    hours_df[col] = hours_df[col].apply(clean_time)
    
hours_df

,monday_opening,monday_closing,tuesday_opening,tuesday_closing,wednesday_opening,wednesday_closing,thursday_opening,thursday_closing,friday_opening,friday_closing,saturday_opening,saturday_closing,sunday_opening,sunday_closing
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0:00,0:00,8:00,18:30,8:00,18:30,8:00,18:30,8:00,18:30,8:00,14:00,NaN,NaN
2,8:00,22:00,8:00,22:00,8:00,22:00,8:00,22:00,8:00,23:00,8:00,23:00,8:00,22:00
3,7:00,20:00,7:00,20:00,7:00,20:00,7:00,20:00,7:00,21:00,7:00,21:00,7:00,21:00
4,NaN,NaN,NaN,NaN,14:00,22:00,16:00,22:00,12:00,22:00,12:00,22:00,12:00,18:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
150341,10:00,19:30,10:00,19:30,10:00,19:30,10:00,19:30,10:00,19:30,10:00,17:30,11:00,17:00
150342,9:30,17:30,9:30,17:30,9:30,17:30,9:30,17:30,9:30,17:30,9:00,17:00,0:00,16:00
150343,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
150344,9:00,20:00,9:00,20:00,9:00,20:00,9:00,20:00,9:00,20:00,9:00,17:00,10:00,17:00


In [13]:
# Concat on axis 1, i.e. combine the DataFrames side by side

for_sql = pd.concat([for_sql, hours_df], axis = 1)

In [14]:
# Checking the transformation

for_sql.head()

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,...,wednesday_opening,wednesday_closing,thursday_opening,thursday_closing,friday_opening,friday_closing,saturday_opening,saturday_closing,sunday_opening,sunday_closing
0,Pns2l4eNsfO8kk83dixA6A,"Abby Rappoport, LAC, CMQ","1616 Chapala St, Ste 2",Santa Barbara,CA,93101,34.426679,-119.711197,5.0,7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,mpf3x-BjTdTEA3yCZrAYPw,The UPS Store,87 Grasso Plaza Shopping Center,Affton,MO,63123,38.551126,-90.335695,3.0,15,...,8:00,18:30,8:00,18:30,8:00,18:30,8:00,14:00,NaN,NaN
2,tUFrWirKiKi_TAnsVWINQQ,Target,5255 E Broadway Blvd,Tucson,AZ,85711,32.223236,-110.880452,3.5,22,...,8:00,22:00,8:00,22:00,8:00,23:00,8:00,23:00,8:00,22:00
3,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,935 Race St,Philadelphia,PA,19107,39.955505,-75.155564,4.0,80,...,7:00,20:00,7:00,20:00,7:00,21:00,7:00,21:00,7:00,21:00
4,mWMc6_wTdE0EUBKIGXDVfA,Perkiomen Valley Brewery,101 Walnut St,Green Lane,PA,18054,40.338183,-75.471659,4.5,13,...,14:00,22:00,16:00,22:00,12:00,22:00,12:00,22:00,12:00,18:00


In [15]:
for_sql.drop(columns = ['hours'], axis = 1, inplace = True)

In [16]:
for_sql.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150346 entries, 0 to 150345
Data columns (total 27 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   business_id        150346 non-null  object 
 1   name               150346 non-null  object 
 2   address            150346 non-null  object 
 3   city               150346 non-null  object 
 4   state              150346 non-null  object 
 5   postal_code        150346 non-null  object 
 6   latitude           150346 non-null  float64
 7   longitude          150346 non-null  float64
 8   stars              150346 non-null  float64
 9   review_count       150346 non-null  int64  
 10  is_open            150346 non-null  int64  
 11  attributes         136602 non-null  object 
 12  categories         150243 non-null  object 
 13  monday_opening     114474 non-null  object 
 14  monday_closing     114474 non-null  object 
 15  tuesday_opening    120631 non-null  object 
 16  tu

### Cleaning the `categories` Column

In order to expand upon the information in the `categories` column, we will create a new column called `primary_category`, which will contain the first value in the `categories` column. Afterwards, we will rename the `categories` column as `secondary_categories`.

In [17]:
# Returns a value of either the first or second value in the categories string

def clean_primaries(cats):
    if cats is not None:
        return cats.split(', ')[0]
    else:
        return np.nan
    
def clean_secondaries(cats):
    if cats is not None:
        
        # Returns the second value after splitting, if a second value exists
        # Otherwise, returns np.nan
        
        if len(cats.split(', ')) > 1:
            return cats.split(', ')[1]
        else: return np.nan
        
    else:
        return np.nan

In [18]:
# Create the primary_category and secondary_categoy columns

for_sql['primary_category'] = for_sql['categories'].apply(clean_primaries)

for_sql['secondary_category'] = for_sql['categories'].apply(clean_secondaries)

In [19]:
# Checking the transformations

for_sql[['categories', 'primary_category', 'secondary_category']].sample(10)

,categories,primary_category,secondary_category
55276,"Nightlife, Cocktail Bars, Restaurants, New Mex...",Nightlife,Cocktail Bars
101116,"Hair Stylists, Beauty & Spas, Hair Salons",Hair Stylists,Beauty & Spas
71727,"Home Decor, Home & Garden, Shopping",Home Decor,Home & Garden
77012,"American (Traditional), Restaurants",American (Traditional),Restaurants
118538,"General Dentistry, Cosmetic Dentists, Endodont...",General Dentistry,Cosmetic Dentists
142693,"Insulation Installation, Gutter Services, Roof...",Insulation Installation,Gutter Services
43169,"Professional Services, Home Services, Structur...",Professional Services,Home Services
59446,"Shoe Stores, Shopping, Fashion, Sporting Goods",Shoe Stores,Shopping
141560,"Restaurants, Steakhouses",Restaurants,Steakhouses
5477,"Home Services, Painters",Home Services,Painters


In [20]:
# Will create a separate table in SQL consistings of the attributes. See attributes_to_sql notebook.

for_sql.drop(
    columns = ['attributes', 'categories'],
    axis = 1,
    inplace = True
)

In [21]:
# Check the drop

for_sql.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150346 entries, 0 to 150345
Data columns (total 27 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   business_id         150346 non-null  object 
 1   name                150346 non-null  object 
 2   address             150346 non-null  object 
 3   city                150346 non-null  object 
 4   state               150346 non-null  object 
 5   postal_code         150346 non-null  object 
 6   latitude            150346 non-null  float64
 7   longitude           150346 non-null  float64
 8   stars               150346 non-null  float64
 9   review_count        150346 non-null  int64  
 10  is_open             150346 non-null  int64  
 11  monday_opening      114474 non-null  object 
 12  monday_closing      114474 non-null  object 
 13  tuesday_opening     120631 non-null  object 
 14  tuesday_closing     120631 non-null  object 
 15  wednesday_opening   123771 non-nul

In [22]:
# Reordering the colums so primary_category and secondary_category are before the opening and closing times.

reorder = for_sql.columns[0:11].to_list() + for_sql.columns[-2::].to_list() + for_sql.columns[11:25].to_list()
for_sql = for_sql[reorder]

In [23]:
# Checking the transformation.

for_sql.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150346 entries, 0 to 150345
Data columns (total 27 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   business_id         150346 non-null  object 
 1   name                150346 non-null  object 
 2   address             150346 non-null  object 
 3   city                150346 non-null  object 
 4   state               150346 non-null  object 
 5   postal_code         150346 non-null  object 
 6   latitude            150346 non-null  float64
 7   longitude           150346 non-null  float64
 8   stars               150346 non-null  float64
 9   review_count        150346 non-null  int64  
 10  is_open             150346 non-null  int64  
 11  primary_category    150243 non-null  object 
 12  secondary_category  149862 non-null  object 
 13  monday_opening      114474 non-null  object 
 14  monday_closing      114474 non-null  object 
 15  tuesday_opening     120631 non-nul

In [26]:
for_sql['is_open'].value_counts()

1    119698
0     30648
Name: is_open, dtype: int64

### Creating a Table in PostgreSQL

In [29]:
# Will need to assign primary key in PGAdmin.

load_dotenv()

engine = sa.create_engine(os.getenv('ENGINE'))

for_sql.to_sql(
    'businesses',
    engine,
    if_exists = 'replace', # This will drop the table if it already exists. Delete this line if necessary.
    index = False,
    dtype = {
        'business_id': sa.types.Text(),
        'name': sa.types.Text(),
        'address': sa.types.Text(),
        'city': sa.types.Text(),
        'state': sa.types.Text(),
        'postal_code': sa.types.Text(), # Text chosen as data type to account for non numeric postal codes
        'latitude': sa.types.FLOAT(),
        'longitude': sa.types.FLOAT(),
        'stars': sa.types.FLOAT(),
        'review_count': sa.types.INTEGER(),
        'is_open': sa.types.Boolean(),
        'monday_opening': sa.types.Time(),
        'monday_closing': sa.types.Time(),
        'tuesday_opening': sa.types.Time(),
        'tuesday_closing': sa.types.Time(),
        'wednesday_opening': sa.types.Time(),
        'wednesday_closing': sa.types.Time(),
        'thursday_opening': sa.types.Time(),
        'thursday_closing': sa.types.Time(),
        'friday_opening': sa.types.Time(),
        'friday_closing': sa.types.Time(),
        'saturday_opening': sa.types.Time(),
        'saturday_closing': sa.types.Time(),
        'sunday_opening': sa.types.Time(),
        'sunday_closing': sa.types.Time(),
    }
)

In [30]:
# Makes business_id the primary key

engine.execute('ALTER TABLE businesses ADD PRIMARY KEY(business_id)')